# Imports

In [ ]:
import wandb
from llm_non_identifiability.runner import LightningGrammarModule
from llm_non_identifiability.datamodule import GrammarDataModule

from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from os.path import abspath, dirname, join
import torch

from copy import deepcopy

# Load model

## Download artifact from wandb

In [ ]:
run = wandb.init()

### Get model

In [ ]:
# model_name = 'model-sw5tu72o:best'
# model = run.use_artifact(f'causal-representation-learning/rule-extrapolation-learning/{model_name}', type='model')
# artifact_dir = model.download()


import wandb
run = wandb.init()
artifact = run.use_artifact('causal-representation-learning/rule_extrapolation/model-lnv4oun6:v0', type='model')
artifact_dir = artifact.download()

In [ ]:
artifact_dir

In [ ]:
artifact

### Get prompt completion tables

In [ ]:
table_name = "run-xgl66xch-id_prompt_completions:v0"
id_prompt_completions_artifact = run.use_artifact(f'causal-representation-learning/llm_non_identifiability/{table_name}', type='run_table')
id_prompt_completions_dir = id_prompt_completions_artifact.download()

In [ ]:
id_prompt_completions: list = id_prompt_completions_artifact.get("id_prompt_completions").data


# Finetuning

## Constants

In [ ]:
MAX_LENGTH = 256
BATCH_SIZE = 128
GRAMMAR = "aNbN"
MAX_EPOCHS = 100

## Load model from checkpoint

In [ ]:
# source: https://pytorch-lightning.readthedocs.io/en/stable/common/checkpointing.html#checkpoint-loading
checkpoint = "model.ckpt"
PATH = join(artifact_dir, checkpoint)

# To load a model along with its weights and hyperparameters use the following method
model: LightningGrammarModule = LightningGrammarModule.load_from_checkpoint(
    PATH, device="cuda" if torch.cuda.is_available() else "cpu", grammar=GRAMMAR
)

## Define datamodule

In [ ]:
datamodule = GrammarDataModule(max_length=MAX_LENGTH, batch_size=BATCH_SIZE, grammar=GRAMMAR)

In [ ]:
datamodule.prepare_data()

In [ ]:
batch = next(iter(datamodule.train_dataloader()))

In [ ]:
X_input = batch[:, :-1]
X_expected = batch[:, 1:]

In [ ]:
pred = model.model(X_input)

In [ ]:
next_items = model._pick_next_tokens(pred)[:, -1].view(-1, 1)

In [ ]:
model._pick_next_tokens(pred)

In [ ]:
X_input.shape

In [ ]:
pred.shape, batch.shape

In [ ]:
(model._pick_next_tokens(pred)== X_input).sum(), (model._pick_next_tokens(pred)!= X_input).sum()

In [ ]:
(model._pick_next_tokens(pred)== X_expected).sum(), (model._pick_next_tokens(pred)!= X_expected).sum()

In [ ]:
model.hparams.loss_fn(pred, X_expected,reduction="none")

In [ ]:
torch.nn.CrossEntropyLoss(reduction="mean")(pred, X_expected)

In [ ]:
torch.nn.CrossEntropyLoss(reduction="mean", ignore_index=2)(pred, X_expected)

In [ ]:
torch.nn.CrossEntropyLoss(reduction="sum")(pred, X_expected)

In [ ]:
torch.nn.CrossEntropyLoss(reduction="sum", ignore_index=2)(pred, X_expected)

## Copy model and change alpha

In [ ]:
bad_model = deepcopy(model)

In [ ]:
bad_model.model.relu_rescale = torch.nn.Parameter(torch.tensor(100.0), requires_grad=False)

## Define logger

In [ ]:
logger = WandbLogger(entity="rule-extrapolation-learning", project="rule-extrapolation", name="finetune-good", log_model="all")

## Checkpoint callback

In [ ]:
model_checkpoint = ModelCheckpoint(
    monitor='Val/loss',
    save_top_k=1,
    mode='min',
)

## Train original model

In [ ]:
trainer  = Trainer(max_epochs=MAX_EPOCHS, logger=logger)
trainer.fit(model, datamodule=datamodule, )


## Train bad model

In [ ]:
assert model.model.relu_rescale.requires_grad == False

In [ ]:
logger = WandbLogger(entity="rule-extrapolation-learning", project="rule-extrapolation", name="finetune-bad", log_model="all")

In [ ]:
trainer  = Trainer(max_epochs=MAX_EPOCHS, logger=logger)
trainer.fit(bad_model, datamodule=datamodule, )